# NYC Motor Vehicle Collision Injury Prediction Pipeline

This notebook implements a reproducible data pipeline that predicts whether a NYC collision results in any injuries or fatalities.

**Research Question:** Given collision context (time, location, vehicle types, contributing factors), can we predict if a crash will cause at least one injury or fatality?

## Section 1: Setup and Configuration


In [9]:
# Core libraries
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path

# Visualization

import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn - preprocessing and pipelines
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

# Sklearn - models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Sklearn - evaluation
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
from dotenv import load_dotenv
import os
load_dotenv()

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_style('whitegrid')

print("All imports successful!")


All imports successful!


In [10]:
# =============================================================================
# CONFIGURATION
# =============================================================================

db_url = os.getenv("FILE_PATH")
# File paths
RAW_DATA_PATH = db_url
DB_PATH = "collisions.db"

# Train/test split configuration (time-based)
TRAIN_YEARS = list(range(2012, 2025))  # 2012-2024
TEST_YEAR = 2025

# Feature configuration
CATEGORICAL_FEATURES = [
    'borough',
    'hour',
    'day_of_week', 
    'is_weekend',
    'is_rush_hour',
    'vehicle_type_1',
    'contributing_factor_1'
]

NUMERIC_FEATURES = [
    'num_vehicles'
]

TARGET = 'severe'

# Vehicle type consolidation mapping (raw values → standardized categories)
VEHICLE_TYPE_MAP = {
    # Sedan/Passenger
    'Sedan': 'Sedan', '4 dr sedan': 'Sedan', '2 dr sedan': 'Sedan',
    'PASSENGER VEHICLE': 'Sedan', '3-Door': 'Sedan',
    # SUV/Station Wagon
    'Station Wagon/Sport Utility Vehicle': 'SUV', 
    'SPORT UTILITY / STATION WAGON': 'SUV',
    # Taxi
    'Taxi': 'Taxi', 'TAXI': 'Taxi', 'Livery Vehicle': 'Taxi',
    # Truck
    'Pick-up Truck': 'Truck', 'Box Truck': 'Truck', 
    'LARGE COM VEH(6 OR MORE TIRES)': 'Truck', 'Tractor Truck Diesel': 'Truck',
    'Flat Bed': 'Truck', 'Dump': 'Truck', 'Tow Truck / Wrecker': 'Truck',
    # Van
    'VAN': 'Van', 'Van': 'Van', 'AMBULANCE': 'Van',
    # Bus
    'Bus': 'Bus', 'BUS': 'Bus',
    # Motorcycle
    'Motorcycle': 'Motorcycle', 'MOTORCYCLE': 'Motorcycle', 'Motorbike': 'Motorcycle',
    # Bike
    'Bike': 'Bike', 'E-Bike': 'Bike', 'E-Scooter': 'Bike',
    # Other/Unknown
    'OTHER': 'Other', 'UNKNOWN': 'Unknown', 'Unknown': 'Unknown'
}

# Contributing factor consolidation mapping
CONTRIBUTING_FACTOR_MAP = {
    'Driver Inattention/Distraction': 'Distraction',
    'Failure to Yield Right-of-Way': 'Failure to Yield',
    'Following Too Closely': 'Following Too Closely',
    'Backing Unsafely': 'Improper Maneuver',
    'Passing or Lane Usage Improper': 'Improper Maneuver',
    'Passing Too Closely': 'Improper Maneuver',
    'Turning Improperly': 'Improper Maneuver',
    'Unsafe Lane Changing': 'Improper Maneuver',
    'Fatigued/Drowsy': 'Fatigue',
    'Traffic Control Disregarded': 'Traffic Violation',
    'Unsafe Speed': 'Speeding',
    'Alcohol Involvement': 'Alcohol/Drugs',
    'Drugs (illegal)': 'Alcohol/Drugs',
    'Driver Inexperience': 'Inexperience',
    'Unspecified': 'Unspecified',
    'Other Vehicular': 'Other'
}

print("Configuration loaded:")
print(f"  Raw data: {RAW_DATA_PATH}")
print(f"  Database: {DB_PATH}")
print(f"  Train years: {TRAIN_YEARS[0]}-{TRAIN_YEARS[-1]}")
print(f"  Test year: {TEST_YEAR}")


Configuration loaded:
  Raw data: Motor_Vehicle_Collisions_-_Crashes_20251202.csv
  Database: collisions.db
  Train years: 2012-2024
  Test year: 2025


## Section 2: Data Loading and Cleaning

Load the raw CSV, parse dates, handle missing values, and create the target variable.


In [11]:
# Load raw data
df_raw = pd.read_csv(RAW_DATA_PATH, low_memory=False)

print(f"Loaded {len(df_raw):,} rows × {len(df_raw.columns)} columns")
print(f"\nColumns: {list(df_raw.columns)}")


Loaded 2,224,642 rows × 29 columns

Columns: ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']


In [12]:
# =============================================================================
# DATA CLEANING
# =============================================================================

df = df_raw.copy()

# 1. Parse date and time
df['crash_datetime'] = pd.to_datetime(
    df['CRASH DATE'] + ' ' + df['CRASH TIME'], 
    format='%m/%d/%Y %H:%M'
)
df['year'] = df['crash_datetime'].dt.year

# 2. Handle missing values
# Borough and ZIP Code: preserve NULLs for KNN imputation (will fill with "Unknown" after imputation)
df['borough'] = df['BOROUGH']
df['zip_code'] = df['ZIP CODE'].astype(str).replace('nan', np.nan)  # Convert string 'nan' to actual NaN

# Coordinates: create flag for missing
df['coords_missing'] = df['LATITUDE'].isna() | df['LONGITUDE'].isna()
df['latitude'] = df['LATITUDE']
df['longitude'] = df['LONGITUDE']

# Contributing factors: fill with "Unknown"
for i in range(1, 6):
    col = f'CONTRIBUTING FACTOR VEHICLE {i}'
    df[f'contributing_factor_{i}'] = df[col].fillna('Unknown') if col in df.columns else 'Unknown'

# Vehicle types: fill with "Unknown"
for i in range(1, 6):
    col = f'VEHICLE TYPE CODE {i}'
    df[f'vehicle_type_{i}'] = df[col].fillna('Unknown') if col in df.columns else 'Unknown'

# 3. Create target variable: SEVERE = 1 if any injury or fatality
df['num_injured'] = df['NUMBER OF PERSONS INJURED'].fillna(0)
df['num_killed'] = df['NUMBER OF PERSONS KILLED'].fillna(0)
df['severe'] = ((df['num_injured'] + df['num_killed']) > 0).astype(int)

# 4. Keep collision ID
df['collision_id'] = df['COLLISION_ID']

print("Data cleaning complete!")
print(f"  Rows: {len(df):,}")
print(f"  Date range: {df['crash_datetime'].min().strftime('%Y-%m-%d')} to {df['crash_datetime'].max().strftime('%Y-%m-%d')}")


Data cleaning complete!
  Rows: 2,224,642
  Date range: 2012-07-01 to 2025-11-29


In [13]:
# =============================================================================
# VALIDATION CHECK: After Data Cleaning
# =============================================================================

print("=" * 70)
print("VALIDATION: After Data Cleaning")
print("=" * 70)

print(f"\nDataset Overview:")
print(f"  Total rows: {len(df):,}")
print(f"  Total columns: {len(df.columns)}")

# Missing data analysis
print(f"\nMissing Data Summary:")
missing_cols = ['borough', 'zip_code', 'latitude', 'longitude']
for col in missing_cols:
    if col in df.columns:
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / len(df)) * 100
        print(f"  {col:15} | Missing: {missing_count:>8,} ({missing_pct:>5.1f}%)")

# Coordinates status
if 'coords_missing' in df.columns:
    missing_coords = df['coords_missing'].sum()
    print(f"  {'coords_missing':15} | Missing: {missing_coords:>8,} ({(missing_coords/len(df)*100):>5.1f}%)")

# Borough distribution
print(f"\nBorough Distribution:")
if 'borough' in df.columns:
    borough_counts = df['borough'].value_counts(dropna=False)
    for borough, count in borough_counts.head(10).items():
        pct = (count / len(df)) * 100
        borough_display = str(borough) if not pd.isna(borough) else 'NULL'
        print(f"  {borough_display:20} | {count:>8,} ({pct:>5.1f}%)")

# Leading Contributing Factors
print(f"\nTop Contributing Factors (Vehicle 1):")
if 'contributing_factor_1' in df.columns:
    cf1_counts = df['contributing_factor_1'].value_counts()
    for factor, count in cf1_counts.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  {str(factor):40} | {count:>8,} ({pct:>5.1f}%)")

print(f"\nTop Contributing Factors (Vehicle 2):")
if 'contributing_factor_2' in df.columns:
    cf2_counts = df['contributing_factor_2'].value_counts()
    for factor, count in cf2_counts.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  {str(factor):40} | {count:>8,} ({pct:>5.1f}%)")

# Leading Vehicle Types
print(f"\nTop Vehicle Types (Vehicle 1):")
if 'vehicle_type_1' in df.columns:
    vt1_counts = df['vehicle_type_1'].value_counts()
    for vtype, count in vt1_counts.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  {str(vtype):40} | {count:>8,} ({pct:>5.1f}%)")

print(f"\nTop Vehicle Types (Vehicle 2):")
if 'vehicle_type_2' in df.columns:
    vt2_counts = df['vehicle_type_2'].value_counts()
    for vtype, count in vt2_counts.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  {str(vtype):40} | {count:>8,} ({pct:>5.1f}%)")

# Target variable distribution
print(f"\nTarget Variable (severe):")
if 'severe' in df.columns:
    severe_counts = df['severe'].value_counts()
    for val, count in severe_counts.items():
        pct = (count / len(df)) * 100
        label = "Injury/Fatal" if val == 1 else "No Injury"
        print(f"  {label:20} ({val}) | {count:>8,} ({pct:>5.1f}%)")

print("\n" + "=" * 70)


VALIDATION: After Data Cleaning

Dataset Overview:
  Total rows: 2,224,642
  Total columns: 50

Missing Data Summary:
  borough         | Missing:  681,099 ( 30.6%)
  zip_code        | Missing:  681,376 ( 30.6%)
  latitude        | Missing:  240,389 ( 10.8%)
  longitude       | Missing:  240,389 ( 10.8%)
  coords_missing  | Missing:  240,389 ( 10.8%)

Borough Distribution:
  NULL                 |  681,099 ( 30.6%)
  BROOKLYN             |  494,784 ( 22.2%)
  QUEENS               |  413,609 ( 18.6%)
  MANHATTAN            |  341,956 ( 15.4%)
  BRONX                |  228,562 ( 10.3%)
  STATEN ISLAND        |   64,632 (  2.9%)

Top Contributing Factors (Vehicle 1):
  Unspecified                              |  744,354 ( 33.5%)
  Driver Inattention/Distraction           |  451,764 ( 20.3%)
  Failure to Yield Right-of-Way            |  133,292 (  6.0%)
  Following Too Closely                    |  119,471 (  5.4%)
  Backing Unsafely                         |   81,174 (  3.6%)
  Other Vehi

In [14]:
# =============================================================================
# BOROUGH & ZIP_CODE IMPUTATION (KNN SPATIAL VOTING)
# =============================================================================

def impute_location_labels_knn(df, k=50, min_neighbors=10):
    """
    Impute missing borough and zip_code using K-Nearest Neighbors spatial voting.
    
    Strategy:
    - Split data into known_locations (has lat, lon, borough, zip) and 
      missing_location_labels (has lat, lon but missing borough and/or zip)
    - Use KNN to find k nearest known neighbors for each missing row
    - Assign borough = mode(neighbor boroughs), zip_code = mode(neighbor zip_codes)
    - If < min_neighbors found, mark as "Unknown"
    
    Args:
        df: DataFrame with 'latitude', 'longitude', 'borough', 'zip_code', 'collision_id'
        k: Number of nearest neighbors to consider (default: 50)
        min_neighbors: Minimum neighbors required for imputation (default: 10)
    
    Returns:
        DataFrame with imputed borough/zip_code and 'imputation_method' column
    """
    df = df.copy()
    
    # Create boolean masks for missing coordinates
    has_coords = df['latitude'].notna() & df['longitude'].notna()
    
    # Split into known and missing location labels
    # Known: has coordinates AND both borough and zip_code are non-null
    known_mask = (
        has_coords & 
        df['borough'].notna() & 
        df['zip_code'].notna() &
        (df['zip_code'] != 'nan')  # Also exclude string 'nan'
    )
    
    # Missing labels: has coordinates BUT missing borough and/or zip_code
    missing_mask = (
        has_coords & 
        (df['borough'].isna() | df['zip_code'].isna() | (df['zip_code'] == 'nan'))
    )
    
    known_locations = df[known_mask].copy()
    missing_location_labels = df[missing_mask].copy()
    
    print("=" * 70)
    print("KNN SPATIAL VOTING IMPUTATION")
    print("=" * 70)
    print(f"Known locations (has lat, lon, borough, zip): {len(known_locations):,}")
    print(f"Missing location labels (has lat, lon, missing borough/zip): {len(missing_location_labels):,}")
    
    if len(known_locations) == 0:
        print("WARNING: No known locations found. Cannot perform imputation.")
        df['imputation_method'] = None
        return df
    
    if len(missing_location_labels) == 0:
        print("No missing location labels to impute.")
        df['imputation_method'] = None
        return df
    
    # Prepare coordinates for KNN
    # Use Euclidean distance on lat/lon (acceptable for NYC-scale distances)
    known_coords = known_locations[['latitude', 'longitude']].values
    missing_coords = missing_location_labels[['latitude', 'longitude']].values
    
    # Fit KNN model on known locations
    print(f"\nFitting KNN model (k={k}) on {len(known_coords):,} known locations...")
    nn = NearestNeighbors(n_neighbors=min(k, len(known_locations)), metric='euclidean', n_jobs=-1)
    nn.fit(known_coords)
    
    # Find neighbors for missing locations (vectorized)
    print(f"Finding neighbors for {len(missing_coords):,} missing location labels...")
    distances, indices = nn.kneighbors(missing_coords)
    
    # Initialize imputation method column
    df['imputation_method'] = None
    
    # Vectorized imputation using mode voting
    print("Performing spatial voting imputation...")
    
    # Get indices of missing rows in original dataframe
    missing_indices = missing_location_labels.index.values
    
    # For each missing row, get its neighbors and compute mode
    imputed_borough = []
    imputed_zip = []
    imputed_method = []
    
    for i, missing_idx in enumerate(missing_indices):
        neighbor_indices = indices[i]
        n_neighbors = len(neighbor_indices)
        
        if n_neighbors < min_neighbors:
            # Not enough neighbors - mark as Unknown
            imputed_borough.append('Unknown')
            imputed_zip.append('Unknown')
            imputed_method.append(None)
        else:
            # Get neighbor boroughs and zip codes
            neighbor_df_indices = known_locations.index[neighbor_indices]
            neighbor_boroughs = df.loc[neighbor_df_indices, 'borough'].values
            neighbor_zips = df.loc[neighbor_df_indices, 'zip_code'].values
            
            # Remove any NaN values from neighbors
            neighbor_boroughs = neighbor_boroughs[~pd.isna(neighbor_boroughs)]
            neighbor_zips = neighbor_zips[~pd.isna(neighbor_zips)]
            neighbor_zips = neighbor_zips[neighbor_zips != 'nan']
            
            # Compute mode (most frequent value)
            if len(neighbor_boroughs) > 0:
                borough_mode = pd.Series(neighbor_boroughs).mode()
                imputed_borough_val = borough_mode.iloc[0] if len(borough_mode) > 0 else 'Unknown'
            else:
                imputed_borough_val = 'Unknown'
            
            if len(neighbor_zips) > 0:
                zip_mode = pd.Series(neighbor_zips).mode()
                imputed_zip_val = zip_mode.iloc[0] if len(zip_mode) > 0 else 'Unknown'
            else:
                imputed_zip_val = 'Unknown'
            
            # Only impute if original value is missing (check for NULLs only)
            original_borough = df.loc[missing_idx, 'borough']
            original_zip = df.loc[missing_idx, 'zip_code']
            
            # Check if borough needs imputation
            borough_needs_imputation = pd.isna(original_borough)
            if borough_needs_imputation:
                imputed_borough.append(imputed_borough_val)
            else:
                imputed_borough.append(original_borough)
            
            # Check if zip_code needs imputation (handle both NULL and string 'nan')
            zip_needs_imputation = pd.isna(original_zip) or (isinstance(original_zip, str) and original_zip == 'nan')
            if zip_needs_imputation:
                imputed_zip.append(imputed_zip_val)
            else:
                imputed_zip.append(original_zip)
            
            # Mark method if we actually imputed something
            if borough_needs_imputation or zip_needs_imputation:
                imputed_method.append('knn_spatial_vote')
            else:
                imputed_method.append(None)
    
    # Update dataframe with imputed values
    df.loc[missing_indices, 'borough'] = imputed_borough
    df.loc[missing_indices, 'zip_code'] = imputed_zip
    df.loc[missing_indices, 'imputation_method'] = imputed_method
    
    # Fill remaining NULLs with "Unknown"
    df['borough'] = df['borough'].fillna('Unknown')
    df['zip_code'] = df['zip_code'].fillna('Unknown').astype(str)
    df['zip_code'] = df['zip_code'].replace('nan', 'Unknown')
    
    return df

# Apply KNN imputation
df = impute_location_labels_knn(df, k=50, min_neighbors=10)

# Validation output
print("\n" + "=" * 70)
print("IMPUTATION VALIDATION")
print("=" * 70)

# Count imputed rows
imputed_mask = df['imputation_method'] == 'knn_spatial_vote'
n_imputed = imputed_mask.sum()

# Count remaining "Unknown" boroughs and zip codes
n_unknown_borough = (df['borough'] == 'Unknown').sum()
n_unknown_zip = (df['zip_code'] == 'Unknown').sum()

# Count original missing (before imputation would have been)
has_coords = df['latitude'].notna() & df['longitude'].notna()
n_with_coords = has_coords.sum()

print(f"\nRows with coordinates: {n_with_coords:,}")
print(f"\nBoroughs:")
print(f"  Filled via KNN: {n_imputed:,}")
print(f"  Remaining 'Unknown': {n_unknown_borough:,} ({n_unknown_borough/len(df)*100:.1f}%)")

print(f"\nZip Codes:")
print(f"  Filled via KNN: {n_imputed:,}")
print(f"  Remaining 'Unknown': {n_unknown_zip:,} ({n_unknown_zip/len(df)*100:.1f}%)")

# Sanity check: Show random sample of imputed rows with their neighbors
if n_imputed > 0:
    print("\n" + "=" * 70)
    print("SANITY CHECK: Sample of 10 Imputed Rows with 5 Nearest Neighbors")
    print("=" * 70)
    
    imputed_rows = df[imputed_mask].sample(min(10, n_imputed), random_state=42)
    
    # For each imputed row, show its 5 nearest neighbors
    known_mask = (
        df['latitude'].notna() & 
        df['longitude'].notna() & 
        df['borough'].notna() & 
        df['zip_code'].notna() &
        (df['zip_code'] != 'nan') &
        (df['zip_code'] != 'Unknown')
    )
    known_locations = df[known_mask]
    known_coords = known_locations[['latitude', 'longitude']].values
    
    nn_sample = NearestNeighbors(n_neighbors=5, metric='euclidean')
    nn_sample.fit(known_coords)
    
    for idx, row in imputed_rows.iterrows():
        print(f"\nImputed Row (collision_id={row['collision_id']}):")
        print(f"  Location: ({row['latitude']:.5f}, {row['longitude']:.5f})")
        print(f"  Imputed Borough: {row['borough']}")
        print(f"  Imputed Zip: {row['zip_code']}")
        
        # Find 5 nearest neighbors
        query_coord = [[row['latitude'], row['longitude']]]
        distances, neighbor_indices = nn_sample.kneighbors(query_coord)
        
        print(f"  5 Nearest Neighbors:")
        for j, (dist, neighbor_idx) in enumerate(zip(distances[0], neighbor_indices[0])):
            neighbor_row = known_locations.iloc[neighbor_idx]
            print(f"    {j+1}. Distance: {dist:.5f} | "
                  f"Borough: {neighbor_row['borough']} | "
                  f"Zip: {neighbor_row['zip_code']} | "
                  f"Coords: ({neighbor_row['latitude']:.5f}, {neighbor_row['longitude']:.5f})")

print("\n" + "=" * 70)
print("KNN IMPUTATION COMPLETE")
print("=" * 70)


KNN SPATIAL VOTING IMPUTATION
Known locations (has lat, lon, borough, zip): 1,505,519
Missing location labels (has lat, lon, missing borough/zip): 478,734

Fitting KNN model (k=50) on 1,505,519 known locations...
Finding neighbors for 478,734 missing location labels...
Performing spatial voting imputation...

IMPUTATION VALIDATION

Rows with coordinates: 1,984,253

Boroughs:
  Filled via KNN: 478,734
  Remaining 'Unknown': 202,616 (9.1%)

Zip Codes:
  Filled via KNN: 478,734
  Remaining 'Unknown': 202,642 (9.1%)

SANITY CHECK: Sample of 10 Imputed Rows with 5 Nearest Neighbors

Imputed Row (collision_id=3849576):
  Location: (40.68195, -73.89652)
  Imputed Borough: BROOKLYN
  Imputed Zip: 11207
  5 Nearest Neighbors:
    1. Distance: 0.00000 | Borough: BROOKLYN | Zip: 11207 | Coords: (40.68195, -73.89652)
    2. Distance: 0.00000 | Borough: BROOKLYN | Zip: 11207 | Coords: (40.68195, -73.89652)
    3. Distance: 0.00000 | Borough: BROOKLYN | Zip: 11207 | Coords: (40.68195, -73.89652)
   

In [15]:
# =============================================================================
# VALIDATION CHECKS
# =============================================================================

print("=" * 60)
print("VALIDATION SUMMARY")
print("=" * 60)

# Target distribution
severe_counts = df['severe'].value_counts()
print(f"\n1. TARGET DISTRIBUTION:")
print(f"   No injury (0): {severe_counts[0]:,} ({severe_counts[0]/len(df)*100:.1f}%)")
print(f"   Injury/fatal (1): {severe_counts[1]:,} ({severe_counts[1]/len(df)*100:.1f}%)")

# Borough distribution
print(f"\n2. BOROUGH DISTRIBUTION:")
for borough, count in df['borough'].value_counts().items():
    print(f"   {borough}: {count:,} ({count/len(df)*100:.1f}%)")

# Year distribution  
print(f"\n3. YEAR DISTRIBUTION:")
year_counts = df['year'].value_counts().sort_index()
for year, count in year_counts.items():
    print(f"   {year}: {count:,}")

# Missing coordinates
missing_coords = df['coords_missing'].sum()
print(f"\n4. COORDINATES:")
print(f"   Missing: {missing_coords:,} ({missing_coords/len(df)*100:.1f}%)")

# Check for any remaining nulls in key columns
key_cols = ['collision_id', 'crash_datetime', 'borough', 'severe']
print(f"\n5. NULL CHECK (key columns):")
for col in key_cols:
    nulls = df[col].isna().sum()
    print(f"   {col}: {nulls} nulls")


VALIDATION SUMMARY

1. TARGET DISTRIBUTION:
   No injury (0): 1,682,159 (75.6%)
   Injury/fatal (1): 542,483 (24.4%)

2. BOROUGH DISTRIBUTION:
   BROOKLYN: 620,065 (27.9%)
   QUEENS: 572,736 (25.7%)
   MANHATTAN: 425,263 (19.1%)
   BRONX: 304,308 (13.7%)
   Unknown: 202,616 (9.1%)
   STATEN ISLAND: 99,654 (4.5%)

3. YEAR DISTRIBUTION:
   2012: 100,545
   2013: 203,742
   2014: 206,046
   2015: 217,708
   2016: 229,831
   2017: 231,007
   2018: 231,564
   2019: 211,486
   2020: 112,917
   2021: 110,557
   2022: 103,887
   2023: 96,607
   2024: 91,314
   2025: 77,431

4. COORDINATES:
   Missing: 240,389 (10.8%)

5. NULL CHECK (key columns):
   collision_id: 0 nulls
   crash_datetime: 0 nulls
   borough: 0 nulls
   severe: 0 nulls


In [16]:
# =============================================================================
# COORDINATE IMPUTATION (K-MEANS)
# =============================================================================

def impute_coordinates_kmeans(df, min_cluster_size=1500):
    """
    Impute missing coordinates using K-means clustering per borough.
    
    For each borough:
    - Fit K-means on rows with valid coordinates
    - Assign borough centroid to rows with missing coordinates
    
    Args:
        df: DataFrame with 'borough', 'latitude', 'longitude', 'coords_missing' columns
        min_cluster_size: Approximate rows per cluster (k = n_rows / min_cluster_size)
    
    Returns:
        DataFrame with imputed coordinates and 'coords_imputed' flag
    """
    df = df.copy()
    df['coords_imputed'] = False
    
    # Get boroughs with valid coordinates (exclude "Unknown")
    valid_boroughs = df[(df['coords_missing'] == False) & (df['borough'] != 'Unknown')]['borough'].unique()
    
    print("Coordinate Imputation by Borough:")
    print("-" * 50)
    
    for borough in valid_boroughs:
        # Masks for this borough
        mask_valid = (df['borough'] == borough) & (df['coords_missing'] == False)
        mask_missing = (df['borough'] == borough) & (df['coords_missing'] == True)
        
        n_valid = mask_valid.sum()
        n_missing = mask_missing.sum()
        
        if n_valid < 10 or n_missing == 0:
            continue
        
        # Determine number of clusters
        n_clusters = max(5, n_valid // min_cluster_size)
        
        # Fit K-means on valid coordinates
        valid_coords = df.loc[mask_valid, ['latitude', 'longitude']].values
        km = KMeans(n_clusters=n_clusters, random_state=42, n_init=5)
        km.fit(valid_coords)
        
        # Compute borough centroid (mean of all valid coords)
        borough_centroid = valid_coords.mean(axis=0)
        
        # Impute missing coords with borough centroid
        df.loc[mask_missing, 'latitude'] = borough_centroid[0]
        df.loc[mask_missing, 'longitude'] = borough_centroid[1]
        df.loc[mask_missing, 'coords_imputed'] = True
        df.loc[mask_missing, 'coords_missing'] = False
        
        print(f"  {borough:15} | Valid: {n_valid:>7,} | Imputed: {n_missing:>6,} | k={n_clusters}")
    
    return df

# Apply imputation
df = impute_coordinates_kmeans(df)

# Summary
print("\n" + "=" * 50)
print("IMPUTATION SUMMARY")
print("=" * 50)
total_imputed = df['coords_imputed'].sum()
still_missing = df['coords_missing'].sum()
print(f"  Coordinates imputed: {total_imputed:,}")
print(f"  Still missing (Unknown borough): {still_missing:,}")


Coordinate Imputation by Borough:
--------------------------------------------------


/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: overflow encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/anime_shh

  BROOKLYN        | Valid: 609,670 | Imputed: 10,395 | k=406


/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: overflow encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/anime_shh

  BRONX           | Valid: 297,351 | Imputed:  6,957 | k=198


...

  MANHATTAN       | Valid: 415,182 | Imputed: 10,081 | k=276


/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: overflow encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/anime_shh

  QUEENS          | Valid: 564,041 | Imputed:  8,695 | k=376


/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/anime_shh/personalCS/data_mgmt/accident-tracker/venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: overflow encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/anime_shh

  STATEN ISLAND   | Valid:  98,009 | Imputed:  1,645 | k=65

IMPUTATION SUMMARY
  Coordinates imputed: 37,773
  Still missing (Unknown borough): 202,616


In [17]:
# =============================================================================
# VALIDATION CHECK: After Location Imputation
# =============================================================================

print("=" * 70)
print("VALIDATION: After Location Imputation (KNN)")
print("=" * 70)

print(f"\nDataset Overview:")
print(f"  Total rows: {len(df):,}")
print(f"  Total columns: {len(df.columns)}")

# Missing data analysis
print(f"\nMissing Data Summary:")
missing_cols = ['borough', 'zip_code', 'latitude', 'longitude']
for col in missing_cols:
    if col in df.columns:
        missing_count = df[col].isna().sum()
        missing_pct = (missing_count / len(df)) * 100
        print(f"  {col:15} | Missing: {missing_count:>8,} ({missing_pct:>5.1f}%)")

# Coordinates status
if 'coords_missing' in df.columns:
    missing_coords = df['coords_missing'].sum()
    print(f"  {'coords_missing':15} | Missing: {missing_coords:>8,} ({(missing_coords/len(df)*100):>5.1f}%)")

# Imputation method summary
if 'imputation_method' in df.columns:
    imputed_count = (df['imputation_method'] == 'knn_spatial_vote').sum()
    print(f"\nImputation Summary:")
    print(f"  Rows imputed via KNN: {imputed_count:,} ({(imputed_count/len(df)*100):>5.1f}%)")

# Borough distribution (after imputation)
print(f"\nBorough Distribution (After Imputation):")
if 'borough' in df.columns:
    borough_counts = df['borough'].value_counts(dropna=False)
    for borough, count in borough_counts.head(10).items():
        pct = (count / len(df)) * 100
        borough_display = str(borough) if not pd.isna(borough) else 'NULL'
        print(f"  {borough_display:20} | {count:>8,} ({pct:>5.1f}%)")

# Leading Contributing Factors
print(f"\nTop Contributing Factors (Vehicle 1):")
if 'contributing_factor_1' in df.columns:
    cf1_counts = df['contributing_factor_1'].value_counts()
    for factor, count in cf1_counts.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  {str(factor):40} | {count:>8,} ({pct:>5.1f}%)")

print(f"\nTop Contributing Factors (Vehicle 2):")
if 'contributing_factor_2' in df.columns:
    cf2_counts = df['contributing_factor_2'].value_counts()
    for factor, count in cf2_counts.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  {str(factor):40} | {count:>8,} ({pct:>5.1f}%)")

# Leading Vehicle Types
print(f"\nTop Vehicle Types (Vehicle 1):")
if 'vehicle_type_1' in df.columns:
    vt1_counts = df['vehicle_type_1'].value_counts()
    for vtype, count in vt1_counts.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  {str(vtype):40} | {count:>8,} ({pct:>5.1f}%)")

print(f"\nTop Vehicle Types (Vehicle 2):")
if 'vehicle_type_2' in df.columns:
    vt2_counts = df['vehicle_type_2'].value_counts()
    for vtype, count in vt2_counts.head(10).items():
        pct = (count / len(df)) * 100
        print(f"  {str(vtype):40} | {count:>8,} ({pct:>5.1f}%)")

# Target variable distribution
print(f"\nTarget Variable (severe):")
if 'severe' in df.columns:
    severe_counts = df['severe'].value_counts()
    for val, count in severe_counts.items():
        pct = (count / len(df)) * 100
        label = "Injury/Fatal" if val == 1 else "No Injury"
        print(f"  {label:20} ({val}) | {count:>8,} ({pct:>5.1f}%)")

print("\n" + "=" * 70)


VALIDATION: After Location Imputation (KNN)

Dataset Overview:
  Total rows: 2,224,642
  Total columns: 52

Missing Data Summary:
  borough         | Missing:        0 (  0.0%)
  zip_code        | Missing:        0 (  0.0%)
  latitude        | Missing:  202,616 (  9.1%)
  longitude       | Missing:  202,616 (  9.1%)
  coords_missing  | Missing:  202,616 (  9.1%)

Imputation Summary:
  Rows imputed via KNN: 478,734 ( 21.5%)

Borough Distribution (After Imputation):
  BROOKLYN             |  620,065 ( 27.9%)
  QUEENS               |  572,736 ( 25.7%)
  MANHATTAN            |  425,263 ( 19.1%)
  BRONX                |  304,308 ( 13.7%)
  Unknown              |  202,616 (  9.1%)
  STATEN ISLAND        |   99,654 (  4.5%)

Top Contributing Factors (Vehicle 1):
  Unspecified                              |  744,354 ( 33.5%)
  Driver Inattention/Distraction           |  451,764 ( 20.3%)
  Failure to Yield Right-of-Way            |  133,292 (  6.0%)
  Following Too Closely                    | 

In [15]:
# =============================================================================
# SELECT COLUMNS FOR CLEANED DATASET
# =============================================================================

# Define columns to keep
columns_to_keep = [
    # Identifiers
    'collision_id',
    'crash_datetime',
    'year',
    
    # Location
    'borough',
    'zip_code',
    'latitude',
    'longitude',
    'coords_missing',
    'coords_imputed',
    
    # Injury counts (for reference)
    'num_injured',
    'num_killed',
    
    # Target
    'severe',
    
    # Vehicle types (1-5)
    'vehicle_type_1', 'vehicle_type_2', 'vehicle_type_3', 
    'vehicle_type_4', 'vehicle_type_5',
    
    # Contributing factors (1-5)
    'contributing_factor_1', 'contributing_factor_2', 'contributing_factor_3',
    'contributing_factor_4', 'contributing_factor_5'
]

df_clean = df[columns_to_keep].copy()

print(f"Cleaned dataset: {len(df_clean):,} rows × {len(df_clean.columns)} columns")
print(f"\nColumns: {list(df_clean.columns)}")
df_clean.head()


Cleaned dataset: 2,224,642 rows × 22 columns

Columns: ['collision_id', 'crash_datetime', 'year', 'borough', 'zip_code', 'latitude', 'longitude', 'coords_missing', 'coords_imputed', 'num_injured', 'num_killed', 'severe', 'vehicle_type_1', 'vehicle_type_2', 'vehicle_type_3', 'vehicle_type_4', 'vehicle_type_5', 'contributing_factor_1', 'contributing_factor_2', 'contributing_factor_3', 'contributing_factor_4', 'contributing_factor_5']


,collision_id,crash_datetime,year,borough,zip_code,latitude,longitude,coords_missing,coords_imputed,num_injured,num_killed,severe,vehicle_type_1,vehicle_type_2,vehicle_type_3,vehicle_type_4,vehicle_type_5,contributing_factor_1,contributing_factor_2,contributing_factor_3,contributing_factor_4,contributing_factor_5
0,4455765,2021-09-11 02:39:00,2021,Unknown,Unknown,NaN,NaN,True,False,2.0,0.0,1,Sedan,Sedan,Unknown,Unknown,Unknown,Aggressive Driving/Road Rage,Unspecified,Unknown,Unknown,Unknown
1,4513547,2022-03-26 11:45:00,2022,Unknown,Unknown,NaN,NaN,True,False,1.0,0.0,1,Sedan,Unknown,Unknown,Unknown,Unknown,Pavement Slippery,Unknown,Unknown,Unknown,Unknown
2,4675373,2023-11-01 01:29:00,2023,BROOKLYN,11230,40.62179,-73.970024,False,False,1.0,0.0,1,Moped,Sedan,Sedan,Unknown,Unknown,Unspecified,Unspecified,Unspecified,Unknown,Unknown
3,4541903,2022-06-29 06:55:00,2022,Unknown,Unknown,NaN,NaN,True,False,0.0,0.0,0,Sedan,Pick-up Truck,Unknown,Unknown,Unknown,Following Too Closely,Unspecified,Unknown,Unknown,Unknown
4,4566131,2022-09-21 13:21:00,2022,Unknown,Unknown,NaN,NaN,True,False,0.0,0.0,0,Station Wagon/Sport Utility Vehicle,Unknown,Unknown,Unknown,Unknown,Passing Too Closely,Unspecified,Unknown,Unknown,Unknown
